In [1]:
import sys
sys.path.append("../..")

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from models.src import train_model

In [11]:
from models.src import evaluate, utils

In [5]:
model_list = ["GaussianNB","HistGradientBoostingClassifier",
                "AdaBoostClassifier",
                "RandomForestClassifier",
                "KNeighborsClassifier",
                "MLPClassifier",
                "SVC",
                "DecisionTreeClassifier"]
param_grid = [
    {}
]*8

In [7]:
outputs = []

In [8]:
for i in tqdm(range(len(model_list))):
    
    model_config = {"model_type": 'traditional', "clf": model_list[i], 
                   "name": f"{model_list[i]}_30sec"}
    data_config = {"use_file": "../../data/features_30_sec.csv", # so no preprocess
                   "split_shuffle": True,
                   "split_stratify": True,
                   "train_split": 0.7, 
                   "test_split": 0.15,
                   "seed": 2024, }
    train_config = {
                    "model_type": "traditional",
                    "grid_search": param_grid[i], #'max_leaf_nodes': [30, 60, 120]},
                    "n_folds": 5, 
                    "metric": ['f1_macro', 'accuracy'], 
                    "seed": 2024,
                    "save_location": "../output"
                    }
    outputs.append(train_model.train(model_config, data_config, train_config,))

 25%|██▌       | 2/8 [00:14<00:49,  8.21s/it]/opt/conda/envs/tensorflow/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/conda/envs/tensorflow/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/conda/envs/tensorflow/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/opt/conda/envs/tensorflow/lib/python3.10/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be 

In [9]:
test_results = pd.DataFrame([outputs[i].results for i in range(len(model_list))])
test_results['model'] = model_list

In [14]:
test_results

,accuracy,f1-macro,worst-performing-label,best-performing-label,model
0,0.440000,0.404916,country,classical,GaussianNB
1,0.833333,0.830145,rock,classical,HistGradientBoostingClassifier
2,0.393333,0.362437,pop,classical,AdaBoostClassifier
3,0.826667,0.823115,rock,classical,RandomForestClassifier
4,0.280000,0.268255,rock,classical,KNeighborsClassifier
5,0.193333,0.158904,disco,classical,MLPClassifier
6,0.286667,0.226399,blues,classical,SVC
7,0.633333,0.630982,rock,classical,DecisionTreeClassifier


In [10]:
cv_results = pd.DataFrame([outputs[i].search.cv_results_ for i in range(len(model_list))])
cv_results['model'] = model_list

In [12]:
cv_results = utils.explode(cv_results, list(set(cv_results.columns)-{'model'}))

In [13]:
cv_results

,model,mean_fit_time,split0_test_f1_macro,split3_test_f1_macro,rank_test_f1_macro,split1_test_f1_macro,split2_test_accuracy,params,std_fit_time,std_score_time,...,split0_test_accuracy,mean_score_time,rank_test_accuracy,std_test_f1_macro,split1_test_accuracy,split4_test_accuracy,mean_test_f1_macro,split4_test_f1_macro,split2_test_f1_macro,split3_test_accuracy
0,GaussianNB,0.003086,0.312964,0.394967,1,0.410748,0.358824,{},0.000153,0.000168,...,0.358824,0.004293,1,0.039268,0.464706,0.423529,0.364796,0.381861,0.323442,0.429412
1,HistGradientBoostingClassifier,2.102022,0.841295,0.754628,1,0.710965,0.800000,{},0.021565,0.000795,...,0.841176,0.019637,1,0.043228,0.717647,0.782353,0.776899,0.782897,0.794708,0.758824
2,AdaBoostClassifier,0.356826,0.129361,0.215088,1,0.284417,0.364706,{},0.003057,0.000488,...,0.205882,0.014641,1,0.072197,0.294118,0.288235,0.239458,0.223714,0.344708,0.311765
3,RandomForestClassifier,0.437039,0.818547,0.751925,1,0.720055,0.788235,{},0.017588,0.001074,...,0.823529,0.010395,1,0.034632,0.723529,0.741176,0.763331,0.741328,0.784800,0.752941
4,KNeighborsClassifier,0.002229,0.305519,0.198340,1,0.335923,0.288235,{},0.000063,0.012492,...,0.311765,0.018586,1,0.045831,0.341176,0.294118,0.279959,0.285284,0.274728,0.211765
5,MLPClassifier,0.348960,0.236935,0.259486,1,0.228409,0.276471,{},0.099517,0.001107,...,0.311765,0.010090,1,0.038894,0.241176,0.223529,0.231308,0.159944,0.271767,0.335294
6,SVC,0.040599,0.255098,0.178705,1,0.189916,0.252941,{},0.003442,0.001209,...,0.305882,0.014166,1,0.029072,0.247059,0.270588,0.201610,0.208372,0.175959,0.229412
7,DecisionTreeClassifier,0.040575,0.603637,0.522432,1,0.594931,0.617647,{},0.001422,0.000132,...,0.605882,0.003976,1,0.033916,0.594118,0.576471,0.582570,0.571618,0.620229,0.529412
